In [1]:
import os
import pandas as pd
from copy import deepcopy
from itertools import combinations, product

In [2]:
scores = pd.read_csv('scores.csv')
data_path = 'generated_result/CP_SS_TS_mask2_71'
new_path = 'generated_result/CP_SS_TS_mask2_71_replace'
os.makedirs(new_path, exist_ok=True)
os.makedirs(f'{new_path}/1_LigandMPNN/seqs/', exist_ok=True)

with open(f'{data_path}/1_LigandMPNN/seqs/0_diffusion_71.fa', 'r') as f:
    sequences = [l.strip('\n') for l in f.readlines() if '>0_diffusion_71' not in l][1:]

In [3]:
replace_map = {
    'F': ['A', 'I', 'L', 'V', 'M'],
    'L': ['V', 'A'],
    'R': ['A'],
    'S': ['A'],
    'Y': ['S', 'A']
}
skip_res = ['H61']

In [8]:
new_sequence = {}
for i, seq_id in enumerate(scores.iloc[:, 0]):
    seq = sequences[seq_id]
    clash_res = scores.loc[i, 'DPP_clash_indices']
    clash_res = clash_res.strip("[]").replace("'", "").split(", ")
    replace_res = [r for r in clash_res if r not in skip_res]
    for n in range(1, len(replace_res)-1):
        for res in combinations(replace_res, n):
            res_rep_candidates = []
            for r in res:
                res_name = r[0]
                if res_name in replace_map:
                    res_rep_candidates.append([f'{res_name}_{r[1:]}_{rep_name}' for rep_name in replace_map[res_name]])
    
            for reps in product(*res_rep_candidates):
                new_seq = deepcopy(seq)
                new_seq = list(new_seq)
                for rep in reps:
                    old, res_id, new = rep.split('_')
                    new_seq[int(res_id) - 1] = new
                new_sequence[f"{seq_id}-{'-'.join(reps)}"] = ''.join(new_seq)

In [9]:
with open(f'{new_path}/1_LigandMPNN/seqs/0_diffusion_71.fa', 'w') as f:
    for key, value in new_sequence.items():
        f.write(f'>{key}\n')
        f.write(f'{value}\n')